In [0]:
import gym
import sys
import pylab
import random
import os
import operator
from collections import deque

from skimage import io, color, transform

import numpy as np
import tensorflow as tf
from keras.models import Model,Sequential
from keras.layers import Flatten,Convolution2D,Input,Dense
from keras.optimizers import Adam
import keras.backend as K

GAME_TYPE = ''
env =gym.make("MsPacman-v0")

In [12]:
#Hyperparameters

#Environment parameters
NUM_EPISODES=100
#We feed the model 4 frames at a time
PHI_LENGTH=4

#Agent parameters
EPSILON=1
EXPERIENCE_REPLAY_CAPACITY=2000
MINIBATCH_SIZE=100
LEARNING_RATE=0.01
ACTION_SIZE=env.action_space.n
EXPLORE=3000000
PREPROCESS_IMAGE_DIM=84 #We downsize the atari frame to 84 x 84

#Adding a parameter loss function to experiment with different losses
ERROR_FUNCTION=tf.losses.huber_loss
print(ACTION_SIZE)

9


In [13]:
'''
Use this cell just to explore the environment, not used towards the algo
'''
state =env.reset()
#print state
print("State.shape = %s" % (state.shape,))
print ("State_size = %s" % (state.size,))
#laying out all features in 1 row, for fun
state = np.reshape(state, [1, state.size])
#print state
print ("State.shape now = %s" % (state.shape,))
print ("State_size now = %s" % (state.size,))
#print env.observation_space
print ("env.observation_space.shape = %s" %(env.observation_space.shape,))

action = random.randrange(env.action_space.n)
next_state, reward, done, info = env.step(action)
#print next_state
print ("next_State.shape = %s" % (next_state.shape,))
next_state = np.reshape(state, [1, next_state.size])
#print next_state
print ("next_State.shape now = %s" % (next_state.shape,))
def preprocess_observation(observation,prediction=False):
    """
    Helper function for preprocessing an observation for consumption by our
    deep learning network
    """
    grayscale_observation = color.rgb2gray(observation)
    processed_image_dim=PREPROCESS_IMAGE_DIM
    resized_observation = transform.resize(grayscale_observation, (processed_image_dim,processed_image_dim)).astype('float32')
    if prediction:
        resized_observation = np.expand_dims(resized_observation, 0)
    return resized_observation

#slight change of notations
#ok let next_state be called x_t1 and current state be x_t
# let reward be r_t and action a_t, 9=env.action_space.n= possible actions
x_t = env.reset()
x_t = preprocess_observation(x_t)
print ("x_t.shape = %s"% (x_t.shape,))

s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
print ("s_t.shape = %s"% (s_t.shape,))
s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*84*84*4
print ("s_t.shape now = %s"% (s_t.shape,))

#above stuff - one time per episode at the start eventualy set s_t = s_t1 and so on 

a_t = random.randrange(env.action_space.n)
x_t1 , r_t, done, info = env.step(a_t)
print (x_t1.shape)

x_t1=preprocess_observation(x_t1)
print ("x_t1.shape = %s"% (x_t1.shape,))
x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1],1) #1x84x84x1
print ("x_t1.shape now = %s"% (x_t1.shape,))
s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)
print ("s_t1.shape = %s"% (s_t1.shape,))
#now s_t =s_t1 and then loop

State.shape = (210, 160, 3)
State_size = 100800
State.shape now = (1, 100800)
State_size now = 100800
env.observation_space.shape = (210, 160, 3)
next_State.shape = (210, 160, 3)
next_State.shape now = (1, 100800)
x_t.shape = (84, 84)
s_t.shape = (84, 84, 4)
s_t.shape now = (1, 84, 84, 4)
(210, 160, 3)
x_t1.shape = (84, 84)
x_t1.shape now = (1, 84, 84, 1)
s_t1.shape = (1, 84, 84, 4)


In [14]:
#visualising deque
D = deque(maxlen=5)
D.append((x_t, a_t, r_t, x_t1, done))
D.append((s_t, a_t, r_t, s_t1, done))
print(D)

deque([(array([[0.3745638 , 0.37254494, 0.30249557, ..., 0.30249557, 0.37254494,
        0.3745638 ],
       [0.53272283, 0.5218396 , 0.14421882, ..., 0.14421882, 0.5218396 ,
        0.53272283],
       [0.5327941 , 0.5217835 , 0.13974088, ..., 0.13974088, 0.5217835 ,
        0.5327941 ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), 3, 0.0, array([[[[0.3745638 ],
         [0.37254494],
         [0.30249557],
         ...,
         [0.30249557],
         [0.37254494],
         [0.3745638 ]],

        [[0.53272283],
         [0.5218396 ],
         [0.14421882],
         ...,
         [0.14421882],
         [0.5218396 ],
         [0.53272283]],

        [[0.5327941 ],
         [0.5217835 ],
         [0.13974088],
         ...,


In [0]:
#lets begin
class Agent:

    def __init__(self, epsilon , experience_replay_capacity , minibatch_size , learning_rate ,action_size, img_dim , explore,loss_func):
        
        self.loss_func=loss_func
        self.action_size=action_size
        self.discount_factor=0.90
        self.learning_rate=learning_rate
        self.epsilon=epsilon
        self.epsilon_min=0.05
        self.batch_size=minibatch_size
        self.train_start=1000
        self.explore=explore
        self.img_channels=4 #phi_length  #coz we feed in 4 stacked b&w imgs instead of 1 rbg img
        self.processed_image_dim=img_dim
        
         # create replay memory using deque
        self.D=deque(maxlen=experience_replay_capacity)
        # create main model and target model
        self.model=self.build_model()
        self.target_model=self.build_model()

        # initialize target model
        self.update_target_model()
        
    def build_model(self):
        
        input_shape=(PHI_LENGTH,PREPROCESS_IMAGE_DIM,PREPROCESS_IMAGE_DIM)
        frame=Input(input_shape)
        #frame is a tensor object
        c1=Convolution2D(32,kernel_size=(8,8),strides=4,activation="relu",padding="same")(frame)
        c2=Convolution2D(64,kernel_size=(4,4),strides=2,activation="relu",padding="same")(c1)
        c3=Convolution2D(64,kernel_size=(3,3),strides=1,activation="relu",padding="same")(c2)
        
        dense_layer=Flatten()(c3)
        
        dense_layer=Dense(512,activation="relu")(dense_layer)
        final=Dense(ACTION_SIZE,activation="linear")(dense_layer)
        
        model=Model(inputs=frame,outputs=final)
        model.compile(loss=self.loss_func,optimizer=Adam(lr=self.learning_rate))
        
        print("finish building the model")
        print(model.summary())
        
        return model
    # after some time interval update the target model to be same with model ###Q-learning specific function
    
    def update_target_model(self):
        #copy weights from the original model to target model
        self.target_model.set_weights(self.model.get_weights())
        
    def append_experience_replay_example(self,s_t,a_t,r_t,s_t1,done):
        """
        Add an experience replay example to our agent's replay memory. If
        memory is full, overwrite previous examples, starting with the oldest
        """
        #D is a memory cell
        #Records State,Action,Reward,Next State and the boolean done
        D.append((s_t, a_t, r_t, s_t1, done))

        if self.epsilon > self.epsilon_min:
            self.epsilon -= (self.epsilon - self.epsilon_min) /self.explore
            #this is just one way of reducing exploration, try out other ways, experiment
        
    def preprocess_observation(self, observation, prediction=False):
        """
        Helper function for preprocessing an observation for consumption by our
        deep learning network
        """
        grayscale_observation = color.rgb2gray(observation)
        resized_observation = transform.resize(grayscale_observation, (self.processed_image_dim, self.processed_image_dim)).astype('float32')
        if prediction:
            resized_observation = np.expand_dims(resized_observation,0)
        return resized_observation
        
    def take_action(self, s_t):
        """
        Given an observation, the model attempts to take an action
        according to its q-function approximation
        """
        #We take an action based on our current epsilon value
        #This is called Epsilon greedy exploration/exploitation
        
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        actions=self.model.predict(s_t)
        return np.argmax(actions[0])
        
    def learn(self):
        """
        Allow the model to collect examples from its experience replay memory
        and learn from them
        """
        minibatch=random.sample(self.D,MINIBATCH_SIZE)
        for s_t,a_t,r_t,s_t1,done in minibatch:
            target=self.model.predict(s_t)
            if done:
                target[0][a_t]=r_t
            else:
                # a = self.model.predict(next_state)[0]
                t=self.target_model.predict(s_t1)[0]
                target[0][a_t]=r_t+self.discount_factor*np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(s_t, target, epochs=1, verbose=0)
            
    def load_model(self,name):
        self.model.load_weights(name)
    
    def save_model(self,name):
        self.model.save_weights(name)

In [0]:
def run_simulation():
    """
    Entry-point for running env simulation
    """

    #print game parameters
    print ("~~~Environment Parameters~~~")
    print ("Num episodes: %s" % NUM_EPISODES)
    print ("Action space: %s" % env.action_space)
    print()
    print ("~~~Agent Parameters~~~")
    print ("Epsilon: %s" % EPSILON)
    print ("Experience Replay Capacity: %s" % EXPERIENCE_REPLAY_CAPACITY)
    print ("Minibatch Size: %s" % MINIBATCH_SIZE)
    print ("Learning Rate: %s" % LEARNING_RATE)

    #initialize agent
    agent = Agent(epsilon=EPSILON,experience_replay_capacity=EXPERIENCE_REPLAY_CAPACITY,minibatch_size=MINIBATCH_SIZE,learning_rate=LEARNING_RATE, action_size =ACTION_SIZE, img_dim =PREPROCESS_IMAGE_DIM ,explore =EXPLORE,loss_func=ERROR_FUNCTION)
    
    scores,episodes=[],[] 
    #initialize auxiliary data structures
    state_list=[] 
    tot_frames=0

    for i_episode in range(NUM_EPISODES):
        print ("Episode: %s" % i_episode)
         
        done = False
        score = 0
        x_t = env.reset()
        x_t = agent.preprocess_observation(x_t)   
        s_t = np.stack((x_t, x_t, x_t, x_t), axis=2) #how many consecutive frames to stack depends on your PHI
        s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  
        
        while not done:
            #env.render()
            #get action for the current state and go one step in environment
            #get action, change score and learn from memory
            
            a_t=agent.take_action(s_t)
            s_t1,r_t,done,_=env.step(a_t)
            score+=r_t if not done else -10
            agent.append_experience_replay_example(s_t,a_t,r_t,s_t1,done)
            s_t=s_t1
            
        if done:
            # every episode update the target model to be same with model
            agent.update_target_model() 
            scores.append(score)
            episodes.append(i_episode)
            print( "  score:", score, "  epsilon:", agent.epsilon)
            
        while True:
            #ensure state list is populated
            if tot_frames < PHI_LENGTH:
                state_list.append(preprocess_observation(x_t))
                tot_frames += 1
                continue



                #update state list with next observation
                state_list.append(preprocess_observation(x_t))
                state_list.pop(0)


In [28]:
if __name__ == "__main__":
    env =gym.make("MsPacman-v0") #env name
    run_simulation()

~~~Environment Parameters~~~
Num episodes: 100
Action space: Discrete(9)

~~~Agent Parameters~~~
Epsilon: 1
Experience Replay Capacity: 2000
Minibatch Size: 100
Learning Rate: 0.01
finish building the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 4, 84, 84)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 1, 21, 32)         172064    
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 1, 11, 64)         32832     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 1, 11, 64)         36928     
_________________________________________________________________
flatten_13 (Flatten)         (None, 704)               0         
_________________________________________________________________
d

KeyboardInterrupt: ignored